In [1]:
import pymn
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import numpy as np
import itertools
import scipy.stats
import seaborn as sns
import scgen 
sc.settings.verbosity = 3   
sc.set_figure_params(figsize = (15,15),facecolor = 'white')

Global seed set to 0
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [17]:
gene_list_all_species = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/duplicated_gene_set_for_miniex.csv', index_col= 0)
gene_list_all_species

sorghum_genes_alpha = gene_list_all_species[['Sorghum','Replace by A']]
sorghum_genes_alpha = sorghum_genes_alpha.dropna()
sorghum_genes_alpha

sorghum_genes_beta = gene_list_all_species[['Sorghum','Replace by B']]
sorghum_genes_beta = sorghum_genes_beta.dropna()
sorghum_genes_beta

setaria_genes_alpha = gene_list_all_species[['Setaria','Replace by A']]
setaria_genes_alpha = setaria_genes_alpha.dropna()
setaria_genes_alpha = setaria_genes_alpha.drop_duplicates(subset= 'Setaria')
setaria_genes_alpha


setaria_genes_beta = gene_list_all_species[['Setaria','Replace by B']]
setaria_genes_beta = setaria_genes_beta.dropna()
setaria_genes_beta = setaria_genes_beta.drop_duplicates(subset= 'Setaria')

setaria_genes_beta


# maize_genes = gene_list_all_species.loc[gene_list_all_species['Species'] == 'M']
# setaria_genes = gene_list_all_species.loc[gene_list_all_species['Species'] == 'Set']
# sorghum_genes

,Setaria,Replace by B
Triplet,,
1,Sevir.3G009000,Zm00001d026665
2,Sevir.3G009500,Zm00001d026662
3,Sevir.3G010300,Zm00001d026657
4,Sevir.3G010800,Zm00001d026653
5,Sevir.3G011200,Zm00001d026652
...,...,...
5047,Sevir.8G041700,Zm00001d004857
5048,Sevir.8G042200,Zm00001d004856
5049,Sevir.8G042300,Zm00001d004855


In [ ]:
setaria_genes_alpha

In [ ]:
alias_File = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/zma_gene_aliases.txt', sep = '\t',)
alias_File['symbol'] = alias_File['symbol'].fillna(alias_File['locus_name'])
alias_File

In [3]:
norm_data = sc.read_h5ad('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/faster_reading_full_data.h5ad')

In [4]:
norm_data.var['Maize True Gene'] = norm_data.var.index

In [5]:
norm_data

AnnData object with n_obs × n_vars = 47536 × 60708
    var: 'Maize True Gene'

In [ ]:
norm_data_maize_genes_to_replace_alpha_sorg = norm_data[:,~norm_data.var.index.isin(sorghum_genes_alpha['Replace by A'])]

In [ ]:
norm_data_maize_genes_to_replace_beta_sorg = norm_data[:,~norm_data.var.index.isin(sorghum_genes_beta['Replace by B'])]

In [18]:
norm_data_maize_genes_to_replace_alpha_set = norm_data[:,~norm_data.var.index.isin(setaria_genes_alpha['Replace by A'])]
norm_data_maize_genes_to_replace_beta_set = norm_data[:,~norm_data.var.index.isin(setaria_genes_beta['Replace by B'])]

In [ ]:
sorghum_genes_alpha

In [ ]:
modified_norm_data_alpha_sorg_names = norm_data_maize_genes_to_replace_alpha_sorg.var.merge(right = sorghum_genes_alpha, left_on = 'Maize True Gene', right_on= 'Sorghum', how = 'left')

In [ ]:
modified_norm_data_beta_sorg_names = norm_data_maize_genes_to_replace_beta_sorg.var.merge(right = sorghum_genes_beta, left_on = 'Maize True Gene', right_on= 'Sorghum', how = 'left')

In [12]:
norm_data_maize_genes_to_replace_beta_set.var

,Maize True Gene
Zm00001d027230,Zm00001d027230
Zm00001d027231,Zm00001d027231
Zm00001d027236,Zm00001d027236
Zm00001d027239,Zm00001d027239
Zm00001d027240,Zm00001d027240
...,...
Sevir.J014000,Sevir.J014000
Sevir.J015100,Sevir.J015100
Sevir.J015900,Sevir.J015900
Sevir.J016300,Sevir.J016300


In [13]:
modified_norm_data_beta_set_names

,Maize True Gene,Setaria,Replace by B
Replace by B,,,
Zm00001d027230,Zm00001d027230,NaN,Zm00001d027230
Zm00001d027231,Zm00001d027231,NaN,Zm00001d027231
Zm00001d027236,Zm00001d027236,NaN,Zm00001d027236
Zm00001d027239,Zm00001d027239,NaN,Zm00001d027239
Zm00001d027240,Zm00001d027240,NaN,Zm00001d027240
...,...,...,...
Sevir.J014000,Sevir.J014000,NaN,Sevir.J014000
Sevir.J015100,Sevir.J015100,NaN,Sevir.J015100
Sevir.J015900,Sevir.J015900,NaN,Sevir.J015900


In [19]:
modified_norm_data_alpha_set_names = norm_data_maize_genes_to_replace_alpha_set.var.merge(right = setaria_genes_alpha, left_on = 'Maize True Gene', right_on= 'Setaria', how = 'left')
modified_norm_data_beta_set_names = norm_data_maize_genes_to_replace_beta_set.var.merge(right = setaria_genes_beta, left_on = 'Maize True Gene', right_on= 'Setaria', how = 'left')

In [ ]:
modified_norm_data_alpha_sorg_names['Replace by A'] = modified_norm_data_alpha_sorg_names['Replace by A'].fillna(value = modified_norm_data_alpha_sorg_names['Maize True Gene'])
modified_norm_data_alpha_sorg_names

In [ ]:
modified_norm_data_beta_sorg_names['Replace by B'] = modified_norm_data_beta_sorg_names['Replace by B'].fillna(value = modified_norm_data_beta_sorg_names['Maize True Gene'])
modified_norm_data_beta_sorg_names

In [20]:
modified_norm_data_alpha_set_names['Replace by A'] = modified_norm_data_alpha_set_names['Replace by A'].fillna(value = modified_norm_data_alpha_set_names['Maize True Gene'])
modified_norm_data_beta_set_names['Replace by B'] = modified_norm_data_beta_set_names['Replace by B'].fillna(value = modified_norm_data_beta_set_names['Maize True Gene'])


In [ ]:
modified_norm_data_alpha_sorg_names.index = modified_norm_data_alpha_sorg_names['Replace by A']

In [ ]:
modified_norm_data_beta_sorg_names.index = modified_norm_data_beta_sorg_names['Replace by B']

In [21]:
modified_norm_data_alpha_set_names.index = modified_norm_data_alpha_set_names['Replace by A']
modified_norm_data_beta_set_names.index = modified_norm_data_beta_set_names['Replace by B']

In [ ]:
modified_norm_data_alpha_sorg_names

In [ ]:
modified_norm_data_alpha_set_names.drop_duplicates(subset = 'Maize True Gene')

In [ ]:
norm_data_swapped_genes_sorghum_alpha = norm_data_maize_genes_to_replace_alpha_sorg.copy()
norm_data_swapped_genes_sorghum_alpha

In [ ]:
norm_data_swapped_genes_sorghum_beta = norm_data_maize_genes_to_replace_beta_sorg.copy()
norm_data_swapped_genes_sorghum_beta

In [22]:
norm_data_swapped_genes_setaria_alpha = norm_data_maize_genes_to_replace_alpha_set.copy()
norm_data_swapped_genes_setaria_beta = norm_data_maize_genes_to_replace_beta_set.copy()


In [ ]:
norm_data_swapped_genes_sorghum_alpha.var = modified_norm_data_alpha_sorg_names


In [ ]:
norm_data_swapped_genes_sorghum_beta.var = modified_norm_data_beta_sorg_names

In [ ]:
norm_data_swapped_genes_setaria_alpha

In [ ]:
modified_norm_data_alpha_set_names

In [23]:
norm_data_swapped_genes_setaria_alpha.var = modified_norm_data_alpha_set_names
norm_data_swapped_genes_setaria_beta.var = modified_norm_data_beta_set_names

In [ ]:
norm_data_swapped_genes_sorghum_alpha.var

In [ ]:
norm_data_swapped_genes_sorghum_beta.var

In [ ]:
#alias_File.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/filled_zma_gene_aliases.txt', index= False, sep = '\t')

In [ ]:
# maize_table = pd.read_csv('/data/CoCoCoNet/geneInfo/maize_info.csv')
# maize_table = maize_table[['EnsemblID','GeneSymbol','UniProtID']]
# maize_table.columns = ['locus_name','symbol','full_name']
# maize_table.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/zma_gene_aliases.txt', index= False, sep = '\t')

In [ ]:
tf_list = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Initialization/zma_TF2fam2mot.txt', sep = '\t')
# tf_list = tf_list['Zm00001d050368']
# tf_list

In [ ]:
tf_list

In [ ]:
tf_list.loc[tf_list['Zm00001d050368'].isin(set_table['locus_name'])]

In [ ]:
#tf_list.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/TF_list.txt', index= False, sep = '\t', header = False)

In [24]:
cell_identity = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/Meta_Maize_Sorghum_Setaria_Nuclei_Cells_SCT_OrthologousgenesGoodname_Sept2022.csv',index_col = 0)
norm_data.obs = cell_identity
# norm_data_swapped_genes_sorghum_alpha.obs = cell_identity
# norm_data_swapped_genes_sorghum_beta.obs = cell_identity

In [25]:
norm_data_swapped_genes_setaria_alpha.obs = cell_identity
norm_data_swapped_genes_setaria_beta.obs = cell_identity

In [ ]:
norm_data.obs = norm_data.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})
norm_data_swapped_genes_sorghum_alpha.obs = norm_data_swapped_genes_sorghum_alpha.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})
norm_data_swapped_genes_sorghum_alpha.obs = norm_data_swapped_genes_sorghum_beta.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})

In [26]:
norm_data_swapped_genes_setaria_alpha.obs = norm_data_swapped_genes_setaria_alpha.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})
norm_data_swapped_genes_setaria_alpha.obs = norm_data_swapped_genes_setaria_beta.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})

In [ ]:
# norm_data.var['ZM'] = norm_data.var_names.str.startswith('Zm')
# norm_data = norm_data[:, norm_data.var['ZM'] == True] 

In [ ]:

norm_data_swapped_genes_sorghum_alpha.var['ZM'] = norm_data_swapped_genes_sorghum_alpha.var_names.str.startswith('Zm')
norm_data_swapped_genes_sorghum_alpha = norm_data_swapped_genes_sorghum_alpha[:, norm_data_swapped_genes_sorghum_alpha.var['ZM'] == True] 


In [ ]:

norm_data_swapped_genes_sorghum_beta.var['ZM'] = norm_data_swapped_genes_sorghum_beta.var_names.str.startswith('Zm')
norm_data_swapped_genes_sorghum_beta = norm_data_swapped_genes_sorghum_beta[:, norm_data_swapped_genes_sorghum_beta.var['ZM'] == True] 


In [27]:

norm_data_swapped_genes_setaria_alpha.var['ZM'] = norm_data_swapped_genes_setaria_alpha.var_names.str.startswith('Zm')
norm_data_swapped_genes_setaria_alpha = norm_data_swapped_genes_setaria_alpha[:, norm_data_swapped_genes_setaria_alpha.var['ZM'] == True] 

norm_data_swapped_genes_setaria_beta.var['ZM'] = norm_data_swapped_genes_setaria_beta.var_names.str.startswith('Zm')
norm_data_swapped_genes_setaria_beta = norm_data_swapped_genes_setaria_beta[:, norm_data_swapped_genes_setaria_beta.var['ZM'] == True] 


In [ ]:
norm_data_swapped_genes_sorghum_alpha

In [ ]:
norm_data_swapped_genes_sorghum_beta

In [ ]:
# cell_type_list = norm_data.obs['Cell Cluster Annotation'].unique()

In [ ]:
# fake_cluster_id = np.arange(len(cell_type_list))
# fake_cluster_id

In [ ]:
# cluster_2_ident = pd.DataFrame(data = zip(fake_cluster_id,cell_type_list))
# cluster_2_ident

In [ ]:
# norm_data.obs['Cell ID']= norm_data.obs.index
# norm_data.obs

In [ ]:
# maize_cells= norm_data[norm_data.obs['Species'] == 'Maize', :]
# sorghum_cells= norm_data[norm_data.obs['Species'] == 'Sorghum', :]
# setaria_cells= norm_data[norm_data.obs['Species'] == 'Setaria', :]

In [ ]:
sorghum_cells_alpha = norm_data_swapped_genes_sorghum_alpha[norm_data_swapped_genes_sorghum_alpha.obs['Species'] == 'Sorghum', :]

In [ ]:
sorghum_cells_beta = norm_data_swapped_genes_sorghum_beta[norm_data_swapped_genes_sorghum_beta.obs['Species'] == 'Sorghum', :]

In [28]:
setaria_cells_alpha = norm_data_swapped_genes_setaria_alpha[norm_data_swapped_genes_setaria_alpha.obs['Species'] == 'Setaria', :]
setaria_cells_beta = norm_data_swapped_genes_setaria_beta[norm_data_swapped_genes_setaria_beta.obs['Species'] == 'Setaria', :]

In [ ]:
# cell_2_clusters = setaria_cells.obs.merge(left_on='Cell Cluster Annotation',right = cluster_2_ident, right_on= 1)
# cell_2_clusters
# cell_2_clusters = cell_2_clusters.rename(columns = {0:'Forced Clusters'})
# cell_2_clusters
# cell_2_clusters = cell_2_clusters[['Cell ID','Forced Clusters','Cell Cluster Annotation']]
# cell_2_clusters
#cell_2_clusters.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/Setaria_only_input/setaria_only_cells2clusters.txt', index= False, sep = '\t', header = False)

In [ ]:
sorghum_cells_alpha

In [ ]:
norm_data.obs

In [ ]:
#cluster_2_ident.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/forced_cluster_identities.txt', index= False, sep = '\t')

In [ ]:
sorghum_cells_alpha.var

In [ ]:
non_duplicated_alpha = sorghum_cells_alpha.var.drop_duplicates(subset = 'Replace by A', keep = False).index
sorghum_cells_alpha = sorghum_cells_alpha[:,sorghum_cells_alpha.var.index.isin(non_duplicated_alpha)]
sorghum_cells_alpha

In [ ]:
non_duplicated_beta = sorghum_cells_beta.var.drop_duplicates(subset = 'Replace by B', keep = False).index
sorghum_cells_beta = sorghum_cells_beta[:,sorghum_cells_beta.var.index.isin(non_duplicated_beta)]
sorghum_cells_beta

In [29]:
non_duplicated_alpha_set = setaria_cells_alpha.var.drop_duplicates(subset = 'Replace by A', keep = False).index
setaria_cells_alpha = setaria_cells_alpha[:,setaria_cells_alpha.var.index.isin(non_duplicated_alpha_set)]
non_duplicated_beta_set = setaria_cells_beta.var.drop_duplicates(subset = 'Replace by B', keep = False).index
setaria_cells_beta = setaria_cells_beta[:,setaria_cells_beta.var.index.isin(non_duplicated_beta_set)]

In [34]:
setaria_cells_beta

View of AnnData object with n_obs × n_vars = 26622 × 29483
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.6', 'seurat_clusters', 'Species', 'Annotation', 'type'
    var: 'Maize True Gene', 'Setaria', 'Replace by B', 'ZM'

In [35]:
setaria_cells_beta.obs

,orig.ident,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Annotation,type
Set_Cell_AAACCTGAGATGGCGT-1,Set_Cells,2524,1465,3060,1459,2,2,Setaria,Cortex_2,Cell
Set_Cell_AAACCTGGTCGTGGCT-1,Set_Cells,5349,1653,4028,1641,4,4,Setaria,Cortex_3,Cell
Set_Cell_AAACGGGAGAGTACAT-1,Set_Cells,6314,2376,4248,2292,11,11,Setaria,Pericycle,Cell
Set_Cell_AAACGGGCATAAAGGT-1,Set_Cells,3653,1467,3627,1449,3,3,Setaria,LRC,Cell
Set_Cell_AAACGGGCATGTCTCC-1,Set_Cells,5384,1956,4208,1939,18,18,Setaria,Epidermis_3,Cell
...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3,9192,2954,21040,3212,14,14,Setaria,Epidermis_2,Cell
Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3,27048,4655,23549,4654,16,16,Setaria,Trichoblast,Cell
Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3,21855,5009,22238,5009,17,17,Setaria,Phloem,Cell
Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3,9015,2669,20346,3006,3,3,Setaria,LRC,Cell


In [ ]:
panda_of_expression_results_sorg_alpha = pd.DataFrame(index = sorghum_cells_alpha.obs.index, columns= sorghum_cells_alpha.var.index, data = sorghum_cells_alpha.X )


In [ ]:
panda_of_expression_results_sorg_beta = pd.DataFrame(index = sorghum_cells_beta.obs.index, columns= sorghum_cells_beta.var.index, data = sorghum_cells_beta.X )


In [30]:
panda_of_expression_results_set_alpha = pd.DataFrame(index = setaria_cells_alpha.obs.index, columns= setaria_cells_alpha.var.index, data = setaria_cells_alpha.X )
panda_of_expression_results_set_beta = pd.DataFrame(index = setaria_cells_beta.obs.index, columns= setaria_cells_beta.var.index, data = setaria_cells_beta.X )


In [ ]:
# panda_of_expression_results_set = pd.DataFrame(index = setaria_cells.obs.index, columns= setaria_cells.var.index, data = setaria_cells.X )
# panda_of_expression_results_maize = pd.DataFrame(index = maize_cells.obs.index, columns= maize_cells.var.index, data = maize_cells.X )
# panda_of_expression_results_sorg = pd.DataFrame(index = sorghum_cells.obs.index, columns= sorghum_cells.var.index, data = sorghum_cells.X )


In [ ]:
filtered_non_expressed_genes_sorg_alpha = panda_of_expression_results_sorg_alpha.loc[:,(panda_of_expression_results_sorg_alpha.sum()>1)]


In [ ]:
filtered_non_expressed_genes_sorg_beta = panda_of_expression_results_sorg_beta.loc[:,(panda_of_expression_results_sorg_beta.sum()>1)]


In [31]:
filtered_non_expressed_genes_set_alpha = panda_of_expression_results_set_alpha.loc[:,(panda_of_expression_results_set_alpha.sum()>1)]
filtered_non_expressed_genes_set_beta = panda_of_expression_results_set_beta.loc[:,(panda_of_expression_results_set_beta.sum()>1)]


In [ ]:
# filtered_non_expressed_genes_set = panda_of_expression_results_set.loc[:,(panda_of_expression_results_set.sum()>1)]
# filtered_non_expressed_genes_maize = panda_of_expression_results_maize.loc[:,(panda_of_expression_results_maize.sum()>1)]
# filtered_non_expressed_genes_sorg = panda_of_expression_results_sorg.loc[:,(panda_of_expression_results_sorg.sum()>1)]



In [ ]:
#filtered_non_expressed_genes

In [ ]:
transposed_matrix_sorg_alpha = filtered_non_expressed_genes_sorg_alpha.T
transposed_matrix_sorg_alpha.index.name = 'geneid'

In [ ]:
transposed_matrix_sorg_beta = filtered_non_expressed_genes_sorg_beta.T
transposed_matrix_sorg_beta.index.name = 'geneid'

In [32]:
transposed_matrix_set_alpha = filtered_non_expressed_genes_set_alpha.T
transposed_matrix_set_alpha.index.name = 'geneid'

transposed_matrix_set_beta = filtered_non_expressed_genes_set_beta.T
transposed_matrix_set_beta.index.name = 'geneid'

In [ ]:
# transposed_matrix_set = filtered_non_expressed_genes_set.T
# transposed_matrix_set.index.name = 'geneid'

# transposed_matrix_maize = filtered_non_expressed_genes_maize.T
# transposed_matrix_maize.index.name = 'geneid'

# transposed_matrix_sorg = filtered_non_expressed_genes_sorg.T
# transposed_matrix_sorg.index.name = 'geneid'

In [ ]:
#transposed_matrix

In [33]:
transposed_matrix_set_alpha.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Setaria_only_input/setariaalpha_matrix.txt', sep = '\t')
transposed_matrix_set_beta.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Setaria_only_input/setariabeta_matrix.txt', sep = '\t')

In [ ]:
transposed_matrix_sorg_beta.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Sorghum_only_input/sorghumbeta_matrix.txt', sep = '\t')

In [ ]:
#transposed_matrix_sorg_alpha.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Sorghum_only_input/sorghum_alpha_matrix.txt', sep = '\t')

In [ ]:
# transposed_matrix_set.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Setaria_only_input/setaria_only_matrix.txt', sep = '\t')
# transposed_matrix_maize.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Maize_only_input/maize_only_matrix.txt', sep = '\t')
# transposed_matrix_sorg.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX_Differnt_gene_list/Sorghum_only_input/sorghum_only_matrix.txt', sep = '\t')

In [ ]:
csv_version_markers = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/setaria_high_purity_markers.csv', index_col= 0)
csv_version_markers

In [ ]:
csv_version_markers = csv_version_markers.merge(left_on='cluster',right = cluster_2_ident, right_on= 1)
csv_version_markers = csv_version_markers.drop(columns= ['cluster',1])
csv_version_markers
csv_version_markers = csv_version_markers.rename(columns = {0:'cluster', 'X':'geneID'})
csv_version_markers
csv_version_markers = csv_version_markers[['geneID','p_val','avg_log2FC','pct.1','pct.2','p_val_adj','cluster','gene']]
csv_version_markers

In [ ]:
#csv_version_markers.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/Setaria_only_input/setaria_only_allMarkers.txt', index= False, sep = '\t')